In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier # using 1NN
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm 
from torchvision.utils import make_grid
from torchvision.utils import save_image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Download the [cow teat datasets](https://github.com/YoushanZhang/SCTL) (10 points) resize image to (224, 224)

In [ ]:
def train_data_loader(train_dirs):
    # Initialize empty lists for to store image arrays and labels
    image_arrays = []
    image_labels = []
   
    # loop to navitage all directories and read information from all the image files
    for label, directory in enumerate(sorted(os.listdir(train_dirs))):
        path = os.path.join(train_dirs, directory)

        # check if it's a directory
        if not os.path.isdir(path):
            continue

        # Iterate over image files in the directory
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)

            # resize and store images
            img = cv2.resize(plt.imread(img_path).copy(), (224, 224))
            image_labels.append(label)
            image_arrays.append(img)
            
    return image_arrays, image_labels


def test_data_loader(data_dir):
    # Get a list of image files in the test data directory
    image_files = [f for f in os.listdir(data_dir) if f.endswith(".jpg")]
    
    # Initialize lists for data and file names
    image_arrays = []
    image_names = []
    
    # Iterate over the image files
    for file in image_files:
        # Get the full path of the image
        path = os.path.join(data_dir, file)
        
        # Read and preprocess the image
        img = cv2.resize(plt.imread(path).copy(), (224, 224))
        #img = img.astype(np.float32) / 255.0
        
        # Append the preprocessed image and file name to the lists
        image_arrays.append(img)
        image_names.append(file)
    
    # Convert lists to NumPy arrays and return
    return image_arrays, image_names

train_loader = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Training'
training_array, training_labels = train_data_loader(train_loader)
training_array, training_labels = np.array(training_array), np.array(training_labels)

test_loader=r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Test_Data'
test_array, test_file_name = test_data_loader(test_loader)
test_array, test_file_name = np.array(test_array), np.array(test_file_name)


### (1). Create a train data loader that returns image arrays and labels

In [2]:
def train_data_loader(train_dirs):
    # Initialize empty lists for to store image arrays and labels
    image_arrays = []
    image_labels = []
   
    # loop to navitage all directories and read information from all the image files
    for label, directory in enumerate(sorted(os.listdir(train_dirs))):
        path = os.path.join(train_dirs, directory)

        # check if it's a directory
        if not os.path.isdir(path):
            continue

        # Iterate over image files in the directory
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)

            # resize and store images
            img = cv2.resize(plt.imread(img_path).copy(), (224, 224))
            image_labels.append(label)
            image_arrays.append(img)
            
    return image_arrays, image_labels

### (2). Create a test data loader that returns image arrays and file names

In [3]:
def test_data_loader(data_dir):
    # Get a list of image files in the test data directory
    image_files = [f for f in os.listdir(data_dir) if f.endswith(".jpg")]
    
    # Initialize lists for data and file names
    image_arrays = []
    image_names = []
    
    # Iterate over the image files
    for file in image_files:
        # Get the full path of the image
        path = os.path.join(data_dir, file)
        
        # Read and preprocess the image
        img = cv2.resize(plt.imread(path).copy(), (224, 224))
        #img = img.astype(np.float32) / 255.0
        
        # Append the preprocessed image and file name to the lists
        image_arrays.append(img)
        image_names.append(file)
    
    # Convert lists to NumPy arrays and return
    return image_arrays, image_names


### (3). Print image arrays, labels and file names dimensions 

### Training Dataset

In [4]:
train_loader = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Training'
training_array, training_labels = train_data_loader(train_loader)
training_array, training_labels = np.array(training_array), np.array(training_labels)

print("Training Dataset Array:",training_array.shape )
print("Training Label Shape:",training_labels.shape )

Training Dataset Array: (1149, 224, 224, 3)
Training Label Shape: (1149,)


In [31]:
training_labels

array([0, 0, 0, ..., 3, 3, 3])

**Observation:**
   - The training dataset contains total 1149 images of size 224, 224 and has 3 channels.

### Test Dataset

In [5]:
# Example usage
test_loader=r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Test_Data'
test_array, test_file_name = test_data_loader(test_loader)
test_array, test_file_name = np.array(test_array), np.array(test_file_name)

print("Test Dataset Array:",test_array.shape )
print("Test File Names Extracted:",test_file_name.shape )


Test Dataset Array: (380, 224, 224, 3)
Test File Names Extracted: (380,)


**Observation:**
   - The test dataset contains total 380 images of size 224, 224 and has 3 channels.

In [71]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(64, 3)  # Replace 'num_classes' with the number of classes in your dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        return x


In [72]:
import torch.optim as optim

# Define the loss function (e.g., CrossEntropy for classification)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (e.g., Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [75]:
# Assuming you have loaded your training data into 'training_array' and 'training_labels'



# Convert data to PyTorch tensors
training_data = torch.Tensor(training_array)
training_labels = torch.LongTensor(training_labels)

# Define the number of classes (replace with the actual number)
num_classes = 10  # Change to your actual number of classes

# Initialize your neural network
model = NeuralNetwork()

# Set the device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)


# Assuming 'training_data' has shape [1149, 224, 224, 3]
# You can permute the dimensions to match the expected shape [1149, 3, 224, 224]
training_data = training_data.permute(0, 3, 1, 2)


# Training loop
num_epochs = 10  # You can adjust this as needed
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(training_data.to(device))
    loss = criterion(outputs, training_labels.to(device))
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pth')


cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 3.44 GiB (GPU 0; 8.00 GiB total capacity; 18.43 GiB already allocated; 0 bytes free; 20.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF